# 1 - Perkenalan
  > Bab pengenalan harus diisi dengan identitas, gambaran besar dataset yang digunakan, dan _objective_ yang ingin dicapai.

## 1 - Identity

  Nama : Jason Rich Darmawan Onggo Putra
  
  Batch : 016 RMT

## 2 - Dataset Overview

## 3 - Objective

# 2 - Import Libraries
  > _Cell_ pertama pada _notebook_ **harus berisi dan hanya berisi** semua _library_ yang digunakan dalam _project_.

# 3 - Data Loading
  > Bagian ini berisi proses penyiapan data sebelum dilakukan eksplorasi data lebih lanjut. Proses Data Loading dapat berupa memberi nama baru untuk setiap kolom, mengecek ukuran dataset, dll.

# 4 - Exploratory Data Analysis (EDA)
  > Bagian ini berisi eksplorasi data pada dataset diatas dengan menggunakan query, grouping, visualisasi sederhana, dan lain sebagainya.

  Task:
  1. insight

# 5 - Data Preprocessing
  > Bagian ini berisi proses penyiapan data untuk proses pelatihan model, seperti pembagian data menjadi train-dev-test, transformasi data (normalisasi, encoding, dll.), dan proses-proses lain yang dibutuhkan.

# 6 - Model Definition
  > Bagian ini berisi cell untuk mendefinisikan model. Jelaskan alasan menggunakan suatu algoritma/model, hyperparameter yang dipakai, jenis penggunaan metrics yang dipakai, dan hal lain yang terkait dengan model.

## 1 - Logistic Regression

## 2 - SVM

## 3 - Decision Tree

## 4 - KKNN

## 5 - Naive Bayes

## 6 - Other Algorithm

# 7 - Model Training
  > Cell pada bagian ini hanya berisi code untuk melatih model dan output yang dihasilkan. Lakukan beberapa kali proses training dengan hyperparameter yang berbeda untuk melihat hasil yang didapatkan. Analisis dan narasikan hasil ini pada bagian Model Evaluation.

## 1 - Default Parameter

### 1 - Logistic Regression

### 2 - SVM

### 3 - Decision Tree

### 4 - KKNN

### 5 - Naive Bayes

### 6 - Other Algorithm

## 2 - Model Tuning

  Goal: To find the best parameters for the algorithm.

### 1 - `sklearn.model_selection.GridSearchCV` (based on best algorithm with default parameters)

  Goal: To find the best parameters for the algorithm.

  Task: 
  1. compare the metrics between algorithm with default parameters and best parameters.

#### 1 - Logistic Regression

#### 2 - SVM

#### 3 - Decision Tree

#### 4 - KKNN

#### 5 - Naive Bayes

#### 6 - Other Algorithm

# 8 - Model Evaluation
  > Pada bagian ini, dilakukan evaluasi model yang harus menunjukkan bagaimana performa model berdasarkan metrics yang dipilih. Hal ini harus dibuktikan dengan visualisasi tren performa dan/atau tingkat kesalahan model. **Lakukan analisis terkait dengan hasil pada model dan tuliskan hasil analisisnya**.

## 1 - `sklearn.model_selection.cross_validate` and `sklearn.metrics`

  Goal: To validate if the metrics aree consistent and not due to data leakage.

  Metrics used are: `sklearn.metrics.precision_score`, `sklearn.metrics.recall_score`, `sklearn.metrics.f1_score` with parameter 'average=None' and `sklearn.metrics.accuracy_score`.

  Task:
  1. metrics interpretation.
  2. weakness/strength of the model.
  3. improvement.

### 1 - Default Parameters

#### 1 - Logistic Regression

#### 2 - SVM

#### 3 - Decision Tree

#### 4 - KKNN

#### 5 - Naive Bayes

#### 6 - Other Algorithm

### 2 - (Algorithm) with Tuned Parameters

  Task:
  1. Only best algorithm gets evaluated.

# 8 - Model Inference
  > Model yang sudah dilatih akan dicoba pada data yang bukan termasuk ke dalam train-set ataupun test-set. Data ini harus dalam format yang asli, bukan data yang sudah di-scaled.

## 1 - (Algorithm) with Tuned Parameters

  Task:
  1. Only best algorithm gets deployed.

# 10 - Pengambilan Kesimpulan
  > Pada bagian terakhir ini, **harus berisi** kesimpulan yang mencerminkan hasil yang didapat dengan _objective_ yang sudah ditulis di bagian pengenalan.

# Conceptual Problems

1. Apakah fungsi parameter `criterion` pada Decision Tree? Jelaskan salah satu criterion yang kalian pahami!
2. Apakah fungsi dari `pruning` pada Tree model?
3. Bagaimana cara memilih `K` yang optimal pada KNN?
4. Jelaskan apa yang kalian ketahui tentang `Cross Validation`!
5. Jelaskan apa yang kalian ketahui tentang `Accuracy, Precision, Recall, F1 Score`!